# 5. Resource Data

In the previous tutorial, we managed to create two resource objects, `project` and `task`, and load them with resource data. In this tutorial, let us explore how to access the resource data.

We use the `tag` resource in the examples so we can print the full `tag` object without a long printout.

In [1]:
#!cat ./aapi_rewrite.py

In [2]:
#!../tools/pylapi_gen aapi_config.py

In [3]:
from pylapi import PyLapi

class aAPI(PyLapi):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.api_url = "https://app.asana.com/api/1.0"
        self._resource_attrs.update({"gid": "$.gid"})

    @PyLapi.resource_method(give="$.data")
    def list(self): pass

    @PyLapi.resource_method(method_route="{gid}", give="$.data", load="$.data")
    def load(self): pass

    @PyLapi.resource_method(method_route="{gid}", http_method="PUT", send={"data": "$"}, give="$.data", load="$.data")
    def update(self): pass

    @PyLapi.resource_method(http_method="POST", send={"data": "$"}, give="$.data", load="$.data")
    def create(self): pass

    @PyLapi.resource_method(method_route="{gid}", http_method="DELETE", give=None)
    def delete(self): pass

Here we use the same root API class and resource classes in [aapy.py](./aAPI.py) and the Asana Personal Access Token (PAT) as in the previous tutorial.

In [4]:
aAPI.auth(open(f"._asecret", "r").readlines()[0].strip())

## Loading Resource Data

Now let us load the resource data of the first `tag` on the list and find the workspace ID the tag belongs to.

In [6]:
@aAPI.resource_class("tag", "tags")
class TagResource(aAPI): pass

In [8]:
tag = aAPI.resource("tag")

tag_list = tag.list()
print(f"tag_list={tag_list}")

# Load the first tag
tag_gid = tag_list[0]["gid"]
tag.load(gid=tag_gid)
print(f"tag={tag}")

tag_list=[{'gid': '1205189493487562', 'name': 'Deliverable', 'resource_type': 'tag'}, {'gid': '1205189493487563', 'name': 'Social', 'resource_type': 'tag'}, {'gid': '1205236784702138', 'name': 'External', 'resource_type': 'tag'}, {'gid': '1205246087901757', 'name': 'PoC', 'resource_type': 'tag'}]
tag={
  "gid": "1205189493487562",
  "color": "dark-orange",
  "created_at": "2023-08-04T22:47:56.631Z",
  "followers": [],
  "name": "Deliverable",
  "notes": "",
  "permalink_url": "https://app.asana.com/0/1205189493487562/1205189493487562",
  "resource_type": "tag",
  "workspace": {
    "gid": "1204597085072493",
    "name": "accsoft.com.au",
    "resource_type": "workspace"
  }
}


Based on the above output, the workspace's gid (Global ID) can be found in `tag.data.workspace.gid`.

In [9]:
workspace_gid = tag.data.workspace.gid
print(workspace_gid)

1204597085072493


Next, we define a new resource class, `workspace`, with `workspaces` as its base route.

In [10]:
@aAPI.resource_class("workspace", "workspaces", gid="$.gid")
class WorkspaceResource(aAPI): pass

To create a resource object, we call the class method `aAPI.resource()` with the resource name `workspace`.

In [11]:
workspace = aAPI.resource("workspace")
print(workspace.load(workspace_gid))

{'gid': '1204597085072493', 'email_domains': ['accsoft.com.au'], 'is_organization': True, 'name': 'accsoft.com.au', 'resource_type': 'workspace'}


The resource data returned by the `load()` method is also loaded into the `workspace` object. Let us print it to verify.

In [12]:
print(workspace)

{
  "gid": "1204597085072493",
  "email_domains": [
    "accsoft.com.au"
  ],
  "is_organization": true,
  "name": "accsoft.com.au",
  "resource_type": "workspace"
}


## Manipulating Resource Objects "Naturally"

You can manipulate the resource object using the familiar attribute and index notation.

Please note that all changes happen locally in the resource object and will not affect any data records stored in the backend storage. If you want to update the backend data, the appropriate API method needs to be implemented. More detail is available in the [Search and Modify](3.%20Search%20and%20Modify.ipynb) tutorial.

In [13]:
print(f"Workspace resource data:\n{workspace}")
print(f"workspace.data.gid: {workspace.data.gid}")
print(f"workspace.data.email_domains: {workspace.data.email_domains}")

Workspace resource data:
{
  "gid": "1204597085072493",
  "email_domains": [
    "accsoft.com.au"
  ],
  "is_organization": true,
  "name": "accsoft.com.au",
  "resource_type": "workspace"
}
workspace.data.gid: 1204597085072493
workspace.data.email_domains: ['accsoft.com.au']


To add a new attribute to the resource data, you simply assign a value to the attribute under `workspace.data`.

In [14]:
workspace.data.new_attribute = "some value"
print(f"After 'new_attribute' is added, workspace becomes: {workspace}")

After 'new_attribute' is added, workspace becomes: {
  "gid": "1204597085072493",
  "email_domains": [
    "accsoft.com.au"
  ],
  "is_organization": true,
  "name": "accsoft.com.au",
  "resource_type": "workspace",
  "new_attribute": "some value"
}


You can assign a new value to the attribute to update it.

In [15]:
workspace.data.new_attribute = "new value"
print(f"After 'new_attribute' is updated, workspace becomes: {workspace}")

After 'new_attribute' is updated, workspace becomes: {
  "gid": "1204597085072493",
  "email_domains": [
    "accsoft.com.au"
  ],
  "is_organization": true,
  "name": "accsoft.com.au",
  "resource_type": "workspace",
  "new_attribute": "new value"
}


To delete an attribute, simply use the `del` command.

In [16]:
del workspace.data.new_attribute
print(f"After 'new_attribute' is deleted, workspace becomes: {workspace}")

After 'new_attribute' is deleted, workspace becomes: {
  "gid": "1204597085072493",
  "email_domains": [
    "accsoft.com.au"
  ],
  "is_organization": true,
  "name": "accsoft.com.au",
  "resource_type": "workspace"
}


You may also use `list` operations for list attributes. For example, you may append a new item to the `email_domains` list and modify list items individually.

In [17]:
workspace.data.email_domains.append("onmyweb.net")
print(f"After an email domain is appended, workspace.data.email_domains becomes: {workspace.data.email_domains}")

After an email domain is appended, workspace.data.email_domains becomes: ['accsoft.com.au', 'onmyweb.net']


In [18]:
workspace.data.email_domains[1] = "onmyweb.net.au"
print(f"After the second email domain is updated, workspace.data.email_domains becomes: {workspace.data.email_domains}")

After the second email domain is updated, workspace.data.email_domains becomes: ['accsoft.com.au', 'onmyweb.net.au']


You may use Python's slice and negative index as usual.

In [19]:
print(f"Last email domain: workspace.data.email_domains[-1]: {workspace.data.email_domains[-1]}")

Last email domain: workspace.data.email_domains[-1]: onmyweb.net.au


Deleting list items can be done in the same way.

In [20]:
del workspace.data.email_domains[-1]
print(f"\nAfter the last email domain is deleted, workspace becomes: {workspace.data}")


After the last email domain is deleted, workspace becomes: {"gid": "1204597085072493", "email_domains": ["accsoft.com.au"], "is_organization": true, "name": "accsoft.com.au", "resource_type": "workspace"}


## Resource Objects Are Subscriptable

All resource objects are subscriptable, meaning that you can access resource data attributes using a text subscript. This is useful if you want to programmatically determine which attribute to access.

In [21]:
print(workspace["$.gid"])
print(workspace["$.email_domains"])

1204597085072493
['accsoft.com.au']


Here the `$` symbol represents the resource data `workspace.data` and can be omitted.

In [22]:
# For simplicity, you may omit the "$." prefix in the subscript.
print(workspace["gid"])
print(workspace["email_domains"])

1204597085072493
['accsoft.com.au']


In [23]:
workspace["$.new_attribute"] = "some value"
print(workspace)

{
  "gid": "1204597085072493",
  "email_domains": [
    "accsoft.com.au"
  ],
  "is_organization": true,
  "name": "accsoft.com.au",
  "resource_type": "workspace",
  "new_attribute": "some value"
}


In [24]:
workspace["$.new_attribute"] = "new value"
print(workspace)

{
  "gid": "1204597085072493",
  "email_domains": [
    "accsoft.com.au"
  ],
  "is_organization": true,
  "name": "accsoft.com.au",
  "resource_type": "workspace",
  "new_attribute": "new value"
}


In [25]:
del workspace["$.new_attribute"]
print(workspace)

{
  "gid": "1204597085072493",
  "email_domains": [
    "accsoft.com.au"
  ],
  "is_organization": true,
  "name": "accsoft.com.au",
  "resource_type": "workspace"
}


In [26]:
workspace["$.email_domains"].append("onmyweb.net")
print(workspace["email_domains"])

['accsoft.com.au', 'onmyweb.net']


In [27]:
workspace["$.email_domains[1]"] = "onmyweb.net.au"
print(workspace["email_domains"])

['accsoft.com.au', 'onmyweb.net.au']


In [28]:
print(workspace["$.email_domains[-1]"])

onmyweb.net.au


In [29]:
del workspace["$.email_domains[-1]"]
print(workspace)

{
  "gid": "1204597085072493",
  "email_domains": [
    "accsoft.com.au"
  ],
  "is_organization": true,
  "name": "accsoft.com.au",
  "resource_type": "workspace"
}


## Printing resource objects and their attributes

You may notice that `{workspace}` gives you a beautified JSON printout, while `{workspace.data}` prints like the usual dict or list.

In fact, the `data` attribute is an `attrDict`, which is a class that implements the attribute notation on a dict or list and therefore is printed like one.

You can obtain the original dict or list type value using the `to_data()` functions.

In [30]:
print(f"workspace: type {type(workspace)}")  # <class '__main__.WorkspaceResource'>
print(workspace)

print("")
print(f"workspace.data: type {type(workspace.data)}")  # <class 'gapi.common.attr_dict.attrDict'>
print(workspace.data)

print("")
print(f"workspace.data.to_data(): type {type(workspace.data.to_data())}")  # <class 'dict'>
print(workspace.data.to_data())

print("")
print("Since workspace.data.to_data() is a proper dict object, you can use json.dumps() to beautify it.")
import json
print(type(workspace.data.to_data()))
print(json.dumps(workspace.data.to_data(), indent=4))


workspace: type <class '__main__.WorkspaceResource'>
{
  "gid": "1204597085072493",
  "email_domains": [
    "accsoft.com.au"
  ],
  "is_organization": true,
  "name": "accsoft.com.au",
  "resource_type": "workspace"
}

workspace.data: type <class 'pylapi.attr_dict.AttrDict'>
{"gid": "1204597085072493", "email_domains": ["accsoft.com.au"], "is_organization": true, "name": "accsoft.com.au", "resource_type": "workspace"}

workspace.data.to_data(): type <class 'dict'>
{'gid': '1204597085072493', 'email_domains': ['accsoft.com.au'], 'is_organization': True, 'name': 'accsoft.com.au', 'resource_type': 'workspace'}

Since workspace.data.to_data() is a proper dict object, you can use json.dumps() to beautify it.
<class 'dict'>
{
    "gid": "1204597085072493",
    "email_domains": [
        "accsoft.com.au"
    ],
    "is_organization": true,
    "name": "accsoft.com.au",
    "resource_type": "workspace"
}


---
In the next tutorial, we will discuss [Search and Modify](./6.%20Search%20and%20Modify.ipynb), including search parameters and API methods to create, update, and delete data records at the backend.

## End of page